In [105]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

dias = 1
horas_manha = 5
horas_tarde = 5

S = 3
D = 3
P = 3

gp = [0, 1, 1] # salas x grandes pequenas
ph = [[0, 1, 0], [1, 1, 0], [0, 0, 1]] # professores x disciplinas
dd = [1, 2, 1] # disciplinas x duração
na = [2, 3, 2] # diciplinas x nº aulas
ts = [0, 1, 0] # diciplinas x tamanho sala

h = {}
for dia in range(dias):
    h[dia] = {}
    for hora in range(horas_manha+horas_tarde):
        h[dia][hora] = {}
        for s in range(S):
            h[dia][hora][s] = {}
            for d in range(D):
                h[dia][hora][s][d] = {}
                for p in range(P):
                    h[dia][hora][s][d][p] = solver.BoolVar("[%i][%i][%i][%i][%i]" % (s, dia, hora, d, p))

# Professor poder dar a disciplina
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            for d in range(D):
                for p in range(P):
                    solver.Add(h[dia][hora][s][d][p] <= ph[p][d])

# Sala tem o tamanho certo
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            for d in range(D):
                for p in range(P):
                    solver.Add(h[dia][hora][s][d][p] * gp[s] <= ts[d])
                    
# Número de aulas correto
for d in range(D):
    solver.Add(sum(h[dia][hora][s][d][p] for dia in range(dias) for hora in range(horas_manha+horas_tarde) for s in range(S) for p in range(P)) == na[d])

# Duração de aula correta
for d in range(D):
    for dia in range(dias):
        for hora in range(horas_manha+horas_tarde):
            for s in range(S):
                for p in range(P):
                    if dd[d] == 2 and hora < horas_manha+horas_tarde-1:
                        solver.Add(h[dia][hora][s][d][p] <= dd[d])
                        solver.Add(h[dia][hora+1][s][d][p] <= dd[d])
                    elif dd[d] == 1:
                        solver.Add(h[dia][hora][s][d][p] <= dd[d])

# Uma aula por hora por professor
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for p in range(P):
            solver.Add(sum(h[dia][hora][s][d][p] for d in range(D)) <= 1)

# A sala não pode estar ocupada por duas aulas ao mesmo tempo
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            solver.Add(sum(h[dia][hora][s][d][p] for d in range(D) for p in range(P)) <= 1)
        
# Um professor por disciplina
for d in range(D):
    for p in range(P):
        solver.Add(sum(h[dia][hora][s][d][p] for dia in range(dias) for hora in range(horas_manha+horas_tarde) for s in range(S)) <= 1)

# A mesma sala para cada disciplina
for s in range(S):
    for d in range(D):
        for p in range(P):
            solver.Add(sum(h[dia][hora][s][d][p] for dia in range(dias) for hora in range(horas_manha+horas_tarde)) <= 1)

In [106]:
r = solver.Solve()

def print_horario():
    # Create a 5D list to store the schedule
    schedule = [[[[[0 for _ in range(P)] for _ in range(D)] for _ in range(S)] for _ in range(horas_manha + horas_tarde)] for _ in range(dias)]

    # Extract the solution from the solver and populate the schedule
    for dia in range(dias):
        for hora in range(horas_manha + horas_tarde):
            for s in range(S):
                for d in range(D):
                    for p in range(P):
                        value = h[dia][hora][s][d][p].solution_value()
                        if value == 1:
                            schedule[dia][hora][s][d][p] = value

    # Print the schedule in a human-readable format
    for dia in range(dias):
        print(f"Day {dia + 1}:")
        for hora in range(horas_manha + horas_tarde):
            print(f"Hour {hora + 1}:", end=" ")
            for s in range(S):
                for d in range(D):
                    for p in range(P):
                        if h[dia][hora][s][d][p].solution_value() == 1:
                            print(f"Room {s + 1}, Discipline {d + 1}, Professor {p + 1}")
            print("\n")
        print()  # Separate days with an empty line

if r == pywraplp.Solver.OPTIMAL:
    print("Solução encontrada") 
    print_horario()
else:
    print("Não foi encontrada solução")

Não foi encontrada solução
